In [8]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import sklearn 
import time 

from sklearn.model_selection import train_test_split, GridSearchCV  
from sklearn.metrics import accuracy_score 
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier 


In [3]:
def get_new_feature_name(df_old_feature_name):
    df_feature_duplicate = pd.DataFrame(data=df_old_feature_name.groupby("column_name").cumcount(), columns=["duplicate_cnt"])
    df_feature_duplicate = df_feature_duplicate.reset_index()
    df_new_feature_name = pd.merge(df_old_feature_name.reset_index(), df_feature_duplicate, how="outer")
    df_new_feature_name["column_name"] = df_new_feature_name[["column_name", "duplicate_cnt"]].apply(lambda x : x[0] + "_" +str(x[1]) if x[1]>0 else x [0], axis=1)
   
    
    df_new_feature_name = df_new_feature_name.drop(["index"], axis=1)
    return df_new_feature_name
    

def get_human_dataset():
    df_feature_name = pd.read_csv("E:\\workSpace_Dataset\\UCI\\UCI HAR Dataset\\features.txt", sep="\s+", header=None, names=['column_index','column_name'])
    df_new_feature_name = get_new_feature_name(df_feature_name) 
    feature_name = df_new_feature_name.iloc[:, 1].values.tolist()
   
    X_train = pd.read_csv("E:\\workSpace_Dataset\\UCI\\UCI HAR Dataset\\train\\X_train.txt", sep="\s+", names = feature_name)
    X_test = pd.read_csv("E:\\workSpace_Dataset\\UCI\\UCI HAR Dataset\\test\\X_test.txt", sep="\s+", names = feature_name)
   
    y_train = pd.read_csv("E:\\workSpace_Dataset\\UCI\\UCI HAR Dataset\\train\\y_train.txt",sep='\s+',header=None,names=['action'])
    y_test = pd.read_csv("E:\\workSpace_Dataset\\UCI\\UCI HAR Dataset\\test\\y_test.txt",sep='\s+',header=None,names=['action'])
   
    return X_train, X_test, y_train, y_test 

In [4]:
X_train, X_test, y_train, y_test = get_human_dataset()

In [5]:
start_time = time.time()

In [6]:
gb_clf = GradientBoostingClassifier(random_state=121)
gb_clf.fit(X_train, y_train)
predict_result = gb_clf.predict(X_test)
accuracy = accuracy_score(y_test, predict_result)

print("Accuracy of GBM {0:.4f}".format(accuracy))
print("Time {0:.1f}".format(time.time() - start_time))

c:\Users\lio22\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy of GBM 0.9389
Time 899.6


In [ ]:
params = {"n_estimators":[100, 500], 
          "learning_rate" : [0.05, 0.1]}

grid_cv = GridSearchCV(gb_clf, param_grid=params, cv=2, verbose=1)
grid_cv.fit(X_train, y_train)

print("best Parameter:", grid_cv.best_params_)
print("best accuracy {0:.4f}".format(grid_cv.best_score_))

In [ ]:
gb_predict_result = grid_cv.best_estimator_.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_predict_result)
print("Accuracy of GBM : {0:.4f}".format(gb_accuracy))